In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [13]:
data_title_basics = pd.read_csv('title.basics.tsv.gz', sep='\t', compression='gzip', na_values=(r'\N'))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15420\1015345584.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_title_basics = pd.read_csv('title.basics.tsv.gz', sep='\t', compression='gzip', na_values=(r'\N'))


In [15]:
# modifier le type from object to date de la colonne "startYear"
data_title_basics['startYear'] = pd.to_datetime(data_title_basics['startYear'], format='%Y', errors='coerce')
# modifier le type from object to bool de la colonne "isAdult"
data_title_basics['isAdult'] = data_title_basics['isAdult'].astype('bool')

In [16]:
data_title_basics.info()
data_title_basics.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11217433 entries, 0 to 11217432
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   tconst          object        
 1   titleType       object        
 2   primaryTitle    object        
 3   originalTitle   object        
 4   isAdult         bool          
 5   startYear       datetime64[ns]
 6   endYear         float64       
 7   runtimeMinutes  object        
 8   genres          object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(6)
memory usage: 695.4+ MB


tconst                   0
titleType                0
primaryTitle            19
originalTitle           19
isAdult                  0
startYear          1416977
endYear           11085659
runtimeMinutes     7677725
genres              500449
dtype: int64

In [17]:
# checker s'il y a des lignes dupliquées
data_title_basics.duplicated().sum()
# supprimer la colonne endYear qui contient plus de 11 millions de valeurs manquantes
data_title_basics.drop(['endYear'], axis= 1, inplace = True) 
data_title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11217433 entries, 0 to 11217432
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   tconst          object        
 1   titleType       object        
 2   primaryTitle    object        
 3   originalTitle   object        
 4   isAdult         bool          
 5   startYear       datetime64[ns]
 6   runtimeMinutes  object        
 7   genres          object        
dtypes: bool(1), datetime64[ns](1), object(6)
memory usage: 609.8+ MB


In [18]:
# checker les valeurs unique, leur nombre pour chaque colonne quand nécessaire > aide à prendre une décision sur les données qui ne nous serviront pas dans l'analyse.
data_title_basics['titleType'].value_counts()


titleType
tvEpisode       8614784
short           1026375
movie            696417
video            300230
tvSeries         272387
tvMovie          148747
tvMiniSeries      57669
tvSpecial         50023
videoGame         40341
tvShort           10459
tvPilot               1
Name: count, dtype: int64

In [20]:
df = data_title_basics[(data_title_basics['titleType'] == 'tvEpisode') | (data_title_basics['titleType'] == 'tvSeries')]
df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
30893,tt0031458,tvEpisode,El huésped del sevillano,El huésped del sevillano,False,1970-01-01,86.0,Musical
34968,tt0035599,tvSeries,Voice of Firestone Televues,Voice of Firestone Televues,False,1943-01-01,15.0,NaN
35169,tt0035803,tvSeries,The German Weekly Review,Die Deutsche Wochenschau,False,1940-01-01,NaN,"Documentary,News"
37597,tt0038276,tvSeries,You Are an Artist,You Are an Artist,False,1946-01-01,15.0,Talk-Show
38431,tt0039120,tvSeries,Americana,Americana,False,1947-01-01,30.0,"Family,Game-Show"
...,...,...,...,...,...,...,...,...
11217427,tt9916846,tvEpisode,Episode #3.18,Episode #3.18,False,2009-01-01,NaN,"Action,Drama,Family"
11217428,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,False,2009-01-01,NaN,"Action,Drama,Family"
11217429,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,False,2010-01-01,NaN,"Action,Drama,Family"
11217430,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,False,2010-01-01,NaN,"Action,Drama,Family"


In [ ]:
# colonne 'titleType': supprimer les lignes qui sont autres que les types film, short, video
types_a_supprimer = ['tvShort', 'tvMo vie', 'tvEpisode', 'tvSeries', 'videoGame', 'tvPilot', 'tvMiniSeries', 'tvSpecial']
data_title_basics.drop(data_title_basics[data_title_basics['titleType'].isin(set(types_a_supprimer))].index, inplace=True)
data_title_basics['titleType'].value_counts()
data_title_basics['titleType'].isnull().sum()
data_title_basics.tail(3)

In [6]:
# colonne 'startyear': checker les valeurs min et max > si début du film supérieur à 2024-12-31, à supprimer car pas de programmation sur des films pas encore sortis
data_title_basics.drop(data_title_basics[data_title_basics['startYear'] > pd.to_datetime('2024-12-31')].index, inplace=True)
data_title_basics['startYear'].max()
# checker le nombre de valeur null et supprimer car ne serviront pas dans l'analyse (leur suppression n'aura pas de fort impact sur les données > 7% de la BDD apres nettoyage précédent de 2020125 rows)
data_title_basics.dropna(subset=['startYear'], inplace=True) 
data_title_basics['startYear'].isnull().sum()

0

In [7]:
# colonne 'runtimeMinutes': check nombre de valeurs nulles et supprimer, puis créer une colonne "metrage" qui va ctégoriser les films en fonction du nbr de minutes
data_title_basics.dropna(subset=['runtimeMinutes'], inplace=True) # supprimer les valeurs nulles
data_title_basics['runtimeMinutes'].isnull().sum()
data_title_basics['runtimeMinutes'] = data_title_basics['runtimeMinutes'].astype('int') # modifier le type de la colonne
data_title_basics['metrage'] = pd.cut(data_title_basics['runtimeMinutes'], # pd.cut > discrétiseldes données continues en catégories discrètes
                       bins=[0, 60, np.inf], # définit les intervalles : < à 60 minutes='court_metrage', >= à 60 minutes = 'long_metrage'.
                       labels=['court_metrage', 'long_metrage'], # étiquettes attribuées aux valeurs
                       right=False)# indique que les intervalles sont semi-ouverts à gauche et fermés à droite. Par exemple, l'intervalle [0, 60) inclut les valeurs de 0 à 59, mais exclut 60
data_title_basics.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1283194 entries, 0 to 11217431
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   tconst          1283194 non-null  object        
 1   titleType       1283194 non-null  object        
 2   primaryTitle    1283191 non-null  object        
 3   originalTitle   1283191 non-null  object        
 4   isAdult         1283194 non-null  bool          
 5   startYear       1283194 non-null  datetime64[ns]
 6   runtimeMinutes  1283194 non-null  int32         
 7   genres          1253982 non-null  object        
 8   metrage         1283194 non-null  category      
dtypes: bool(1), category(1), datetime64[ns](1), int32(1), object(5)
memory usage: 75.9+ MB


In [8]:
# colonne 'genre': supprimer les valeurs nulles
data_title_basics.dropna(subset=['genres'], inplace=True)
data_title_basics['genres'].isnull().sum()
data_title_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1253982 entries, 0 to 11217431
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   tconst          1253982 non-null  object        
 1   titleType       1253982 non-null  object        
 2   primaryTitle    1253979 non-null  object        
 3   originalTitle   1253979 non-null  object        
 4   isAdult         1253982 non-null  bool          
 5   startYear       1253982 non-null  datetime64[ns]
 6   runtimeMinutes  1253982 non-null  int32         
 7   genres          1253982 non-null  object        
 8   metrage         1253982 non-null  category      
dtypes: bool(1), category(1), datetime64[ns](1), int32(1), object(5)
memory usage: 74.1+ MB


In [9]:
# colonne 'primaryTitle' & 'originalTitle': check des valeurs nulles
data_title_basics.dropna(subset=['primaryTitle'], inplace=True)
data_title_basics['primaryTitle'].isnull().sum()
data_title_basics.dropna(subset=['originalTitle'], inplace=True)
data_title_basics['originalTitle'].isnull().sum()
data_title_basics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1253979 entries, 0 to 11217431
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   tconst          1253979 non-null  object        
 1   titleType       1253979 non-null  object        
 2   primaryTitle    1253979 non-null  object        
 3   originalTitle   1253979 non-null  object        
 4   isAdult         1253979 non-null  bool          
 5   startYear       1253979 non-null  datetime64[ns]
 6   runtimeMinutes  1253979 non-null  int32         
 7   genres          1253979 non-null  object        
 8   metrage         1253979 non-null  category      
dtypes: bool(1), category(1), datetime64[ns](1), int32(1), object(5)
memory usage: 74.1+ MB


In [10]:
data_title_basics.reset_index(drop=True, inplace=True) # reinitialiser les index en cas de multi-index ou souci d'indexation lors du nettoyage
data_title_basics.index #  vérifier les index
data_title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253979 entries, 0 to 1253978
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   tconst          1253979 non-null  object        
 1   titleType       1253979 non-null  object        
 2   primaryTitle    1253979 non-null  object        
 3   originalTitle   1253979 non-null  object        
 4   isAdult         1253979 non-null  bool          
 5   startYear       1253979 non-null  datetime64[ns]
 6   runtimeMinutes  1253979 non-null  int32         
 7   genres          1253979 non-null  object        
 8   metrage         1253979 non-null  category      
dtypes: bool(1), category(1), datetime64[ns](1), int32(1), object(5)
memory usage: 64.6+ MB


In [95]:
# Graphe 2 : Evolution du nombre de films sur les 5 dernières années
#data_5yrs = data_title_basics[(data_title_basics['startYear'] >= '2020-01-01') & (data_title_basics['startYear'] <= '2024-12-31')]
data_from2000 = data_title_basics[(data_title_basics['startYear'] >= '2000-01-01')]
yearly_counts = data_from2000['startYear'].dt.year.value_counts().reset_index()
yearly_counts.columns = ['Year', 'Count']

fig = px.bar(yearly_counts, x='Year', y='Count', color_discrete_sequence=["green"], title='Nombre total de films sortis depuis 2000')
fig.show()

In [ ]:
# Graphe 2 : Nombre de films par genre
genres_exploded = data_title_basics['genres'].str.split(',').explode('genres')  # "exploser" la liste de genres pour créer une nouvelle ligne pour chaque genre associé au film. 
                                                # Cela permettra d'avoir un DataFrame où chaque ligne correspond à un film et à un seul genre.
genres_counts = pd.DataFrame(genres_exploded) # afficher sous forme de dataframe les valeurs explosées
gc = dict(genres_counts['genres'].value_counts()) # afficher les valeurs unique et leur nombre sous forme de dictionnaire
genre_counts = {'Short': 674505, # utiliser le dictionnaire précédemment afficher afin de créer un dataframe contenant 2 colonnes: genre et nombre de film par genre
    'Drama': 393103,
    'Documentary': 242733,
    'Comedy': 230967,
    'Adult': 84481,
    'Horror': 74447,
    'Romance': 73043,
    'Animation': 65512,
    'Action': 62903,
    'Thriller': 55893,
    'Crime': 50169,
    'Family': 44042,
    'Music': 42201,
    'Fantasy': 40430,
    'Adventure': 40053,
    'Mystery': 34750,
    'Sci-Fi': 32521,
    'Biography': 29552,
    'History': 23514,
    'Musical': 15022,
    'Sport': 12787,
    'War': 10920,
    'Western': 9506,
    'News': 6923,
    'Talk-Show': 2044,
    'Reality-TV': 1623,
    'Film-Noir': 871,
    'Game-Show': 212
 }
new_genre_count = pd.DataFrame.from_dict(genre_counts, orient='index', columns=['Count']) # utiliser le dictionnaire précédemment afficher afin de 
                                        # créer un dataframe contenant 2 colonnes: genre et nombre de film par genre
new_genre_count


In [96]:
fig = px.bar(new_genre_count, x=new_genre_count.index, y=new_genre_count['Count'], color_discrete_sequence=["blue"], title="Nombre de films par genre",
            labels={"x": "Genre", "y": "Nombre de films"})
fig.show()


In [106]:
data_title_basics['tconst']

0          tt0000001
1          tt0000002
2          tt0000003
3          tt0000004
4          tt0000005
             ...    
1253974    tt9916680
1253975    tt9916724
1253976    tt9916730
1253977    tt9916754
1253978    tt9916856
Name: tconst, Length: 1253979, dtype: object

In [ ]:
title_basics_exploded = data_title_basics.explode('genres')

crosstab = title_basics_exploded.pivot_table(index='tconst', columns='genres')

title_basics_exploded

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,metrage
0,tt0000001,short,Carmencita,Carmencita,False,1894-01-01,1,"Documentary,Short",court_metrage
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892-01-01,5,"Animation,Short",court_metrage
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,False,1892-01-01,5,"Animation,Comedy,Romance",court_metrage
3,tt0000004,short,Un bon bock,Un bon bock,False,1892-01-01,12,"Animation,Short",court_metrage
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,False,1893-01-01,1,"Comedy,Short",court_metrage
...,...,...,...,...,...,...,...,...,...
1253974,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,False,2007-01-01,100,Documentary,long_metrage
1253975,tt9916724,short,Hay Que Ser Paciente,Hay Que Ser Paciente,False,2015-01-01,3,"Documentary,Short",court_metrage
1253976,tt9916730,movie,6 Gunn,6 Gunn,False,2017-01-01,116,Drama,long_metrage
1253977,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,False,2013-01-01,49,Documentary,court_metrage


In [ ]:
crosstab.plot.bar(stacked=True)
plt.title('Répartition des films par genres')
plt.xlabel('Films')
plt.ylabel('Genres')
plt.show()
